In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
from konlpy.tag import Hannanum
from konlpy.tag import Okt
from konlpy.tag import Komoran
import sqlite3 as sq3
from pandas.io import sql
import os
import numpy as np
import math

## read db file

In [2]:
# 데이터베이스 저장
def db_save(data, db_name, table_name):
    with sq3.connect(os.path.join('.',db_name)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            data.to_sql(name = table_name, con = con, index = False, if_exists='replace') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(data), '건 저장완료..')

In [3]:
# 데이터베이스 불러오기
def db_select(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return df  

### Save dictionaries

In [4]:
import pickle

In [5]:
def saveDict(my_dict, file_name):
    with open(file_name, "wb") as tf:
        pickle.dump(my_dict,tf)
    print(len(my_dict),' saved to file...')

In [6]:
def readDict(file_name):
    new_dict = dict()
    with open(file_name, "rb") as tf:
        new_dict = pickle.load(tf)
    return(new_dict)

# save word tokens into a file

In [7]:
DB_NAME = '법무법인_태평양_법률자료.db'
TABLE_NAME = '태평양_tokenized'
df = db_select(DB_NAME,TABLE_NAME)
df1 = df.iloc[:,[0,4]].copy()
df1

,제목,BoW
0,한국-인도네시아 포괄적 경제동반자협정(CEPA) 비준,"인도네시아,국회,한인니,비준,진행,상황,인도네시아,국회,한인니,비준,진행,상황,인도..."
1,지역주택조합이 조합총회 결의 없이 체결한 계약의 효력에 관한 대법원판결(대법원 20...,"주택법,시행규칙,지역주택조합,예산,조합원,부담,계약,체결,총회,의결,주택법,시행규칙..."
2,금융투자업자 간 업무용으로 지정되지 않은 정보통신수단을 사용하여 이루어진 매매 교섭...,"금융투자업자,주문기록,유지의무,정보통신수단,지정,금융투자업자,주문기록,유지의무,정보..."
3,도심복합사업 개편 – 정부의 국민 주거안정 실현방안 발표 및 ‘도심 복합개발 지원에...,"정부,국회,주택,공급,공공주택,특별법,개정,공공주택,복합사업,도입,사업,공공,주도적..."
4,2022년도 지방세입 관계법률 개정안 주요 사항,"행정안전부,지방세발전위원회,개최,경제,활력,민생안정,지원,지방세입,관계법률,개정안,..."
...,...,...
357,가맹사업법 시행령 개정안과 관련하여 알려드립니다.,"가맹사업법,시행령,개정안,가맹사업법,시행령,개정안,가맹사업법,시행령,개정안,가맹사업..."
358,부당지원행위 등 공정거래법 시행령 개정 및 부당특약 등 하도급법 시행령 개정과 관련...,"부당지원행위,공정거래법,시행령,개정,부당지원행위,공정거래법,시행령,개정,부당지원행위..."
359,환경오염물질 배출시설 관리체계 전면 개편,"환경오염물질,배출시설,관리체계,전면,개편,환경오염물질,배출시설,관리체계,전면,개편,..."
360,상해자유무역구 부가가치통신업무 개방,"상해자유무역구,개방,상해자유무역구,개방,상해자유무역구,개방,상해자유무역구,개방,중국..."


# 단어의 총 등장 횟수 구하기 --> TF

In [8]:
def dict_each(string):
    each_art = dict()     # dictionary for each article
    temp_list = []        # temporary list for the word tokens
    
    for i in string.split(','):
        temp_list.append(i)

    for each_t in temp_list:
        if each_t in each_art.keys():
            each_art[each_t] += 1
        else :
            each_art[each_t] = 1
            
    t_list = sorted(each_art.items(), key=lambda x:x[1], reverse=True)
    each_TF = dict(t_list)
    return each_TF

In [9]:
total_TF = []

for i in df1.iloc[:,1]:
    tokens_const = []
    total_TF.append(dict_each(i))

len(total_TF)

362

In [10]:
saveDict(total_TF,'TF_file.pkl')
#saveDict(total_TF,'TF-IDF_file.pkl')

362  saved to file...


In [11]:
TF_new = readDict('TF_file.pkl')
DF = readDict('DF_file.pkl')

# TF-IDF
##### 참고: https://brunch.co.kr/@bflysoft1117/127, https://www.researchgate.net/publication/342833072_gujojeog_topig_modelling-eul_hwal-yonghan_gong-gongdeiteo_suyo_bunseog_Analysis_of_Open_Government_Data_Demand_Using_Structural_Topic_Modeling
높을수록 단어 중요도가 높다
### TF-IDF = TF X log(N/DF)
- TF: 문서 내 특정 단어의 빈도 수
- N: 분석 대상 문서 통합
- DF: 특정 단어를 포함하는 문서빈도 수

In [12]:
def TF_IDF_each(dct):
    TF_IDF_i = dict()     # dictionary to save TF-IDF values
    
    for eachkey in list(dct.keys()):
        TF_temp = dct[eachkey]
        DF_temp = DF[eachkey]
        TF_IDF_1 = TF_temp*math.log(N/DF_temp)
        TF_IDF_i[eachkey] = TF_IDF_1

    temp_tf_idf = sorted(TF_IDF_i.items(), key=lambda x:x[1], reverse=True)
    TF_IDF_temp = dict(temp_tf_idf)
    return TF_IDF_temp

In [13]:
TF_IDF = []     # dictionary to save TF-IDF values

N = len(df1)
for each_article in total_TF:
    each_TI = TF_IDF_each(each_article)
    TF_IDF.append(each_TI)

# Find the 10 keywords of the first article
- ordered by importance

In [14]:
def get_each_keywords(dictionary):
    keyword_s = ""
    
    temp_list = sorted(dictionary.items(), key=lambda x:x[1], reverse=True)
    temp_2 = dict(temp_list)
    keywords = list(temp_2.keys())[:10]
    keyword_s = ','.join(keywords)
    
    return keyword_s

In [15]:
TF_IDF_final = []

for each in TF_IDF:
    TF_IDF_final.append(get_each_keywords(each))
    
df['키워드10개'] = TF_IDF_final
df

,제목,날짜,관련주제,본문내용,BoW,키워드10개
0,한국-인도네시아 포괄적 경제동반자협정(CEPA) 비준,2022.09.13,"기업법무,해외투자,국제쟁송,관세·국제통상,동남아시아",인도네시아 국회의 한인니 CEPA 비준 및 진행 상황 인도네시아 국회의 한인니 C...,"인도네시아,국회,한인니,비준,진행,상황,인도네시아,국회,한인니,비준,진행,상황,인도...","한인니,인도네시아,원산지,비준,한인니CEPA,엔지니어링,발효,국회,관세,통관절차"
1,지역주택조합이 조합총회 결의 없이 체결한 계약의 효력에 관한 대법원판결(대법원 20...,2022.09.06,"국내소송,건설·부동산소송,건설",주택법 시행규칙은 지역주택조합이 예산으로 정한 사항 외에 조합원에게 부담이 될 계약...,"주택법,시행규칙,지역주택조합,예산,조합원,부담,계약,체결,총회,의결,주택법,시행규칙...","지역주택조합,주택법,총회,조합원,법률행위,계약,비법인사단,예산,흠결,절차적"
2,금융투자업자 간 업무용으로 지정되지 않은 정보통신수단을 사용하여 이루어진 매매 교섭...,2022.09.05,"자본시장,국내소송,증권금융소송,금융회사,은행,증권,자산운용,비은행 금융기관",배경 배경 배경 1 금융투자업자의 주문기록 유지의무 및 업무용 정보통신수단 지정 ...,"금융투자업자,주문기록,유지의무,정보통신수단,지정,금융투자업자,주문기록,유지의무,정보...","정보통신수단,원고들,지급거절,주문기록,자산담보부기업어음ABCP,금융투자업자,임직원들..."
3,도심복합사업 개편 – 정부의 국민 주거안정 실현방안 발표 및 ‘도심 복합개발 지원에...,2022.08.23,"국내소송,건설·부동산소송,건설",정부와 국회는 도심 내 주택을 신속히 공급하기 위해 공공주택 특별법을 개정하여 도심...,"정부,국회,주택,공급,공공주택,특별법,개정,공공주택,복합사업,도입,사업,공공,주도적...","성장거점형,주거중심형,용적률,토지주,도심복합사업,사업시행자,민간,첨단산업,고밀,광역..."
4,2022년도 지방세입 관계법률 개정안 주요 사항,2022.08.12,조세,행정안전부는 2022 8 11 지방세발전위원회를 개최하여 경제 활력 제고와 민생안정...,"행정안전부,지방세발전위원회,개최,경제,활력,민생안정,지원,지방세입,관계법률,개정안,...","취득세,감면,납세의무,과세표준,지방세,심판청구,간접투자회사,세율,개정안,국세"
...,...,...,...,...,...,...
357,가맹사업법 시행령 개정안과 관련하여 알려드립니다.,2014.02.13,공정거래 조사·소송,가맹사업법 시행령 개정안과 관련하여 가맹사업법 시행령 개정안과 관련하여 가맹사업법 ...,"가맹사업법,시행령,개정안,가맹사업법,시행령,개정안,가맹사업법,시행령,개정안,가맹사업...","가맹사업법,가맹본부,점포환경,개정안,상황정보,시행령,구체화시행령,변경등록,정보공개,오전"
358,부당지원행위 등 공정거래법 시행령 개정 및 부당특약 등 하도급법 시행령 개정과 관련...,2014.02.11,공정거래 조사·소송,부당지원행위 등 공정거래법 시행령 개정 부당지원행위 등 공정거래법 시행령 개정 부당...,"부당지원행위,공정거래법,시행령,개정,부당지원행위,공정거래법,시행령,개정,부당지원행위...","부당지원행위,공정거래법,시행령,특수관계인,유리한,하도급법,객체,개정,원사업자,사업기회"
359,환경오염물질 배출시설 관리체계 전면 개편,2014.02.06,"환경,환경",환경오염물질 배출시설 관리체계 전면 개편 환경오염물질 배출시설 관리체계 전면 개편 ...,"환경오염물질,배출시설,관리체계,전면,개편,환경오염물질,배출시설,관리체계,전면,개편,...","환통법,배출시설,입법예고안,통합허가,입지규제,변경허가,환경부,오염물질,기술규제,업종별"
360,상해자유무역구 부가가치통신업무 개방,2014.02.04,중국,상해자유무역구 부가가치통신업무 개방 상해자유무역구 부가가치통신업무 개방 상해자유무역...,"상해자유무역구,개방,상해자유무역구,개방,상해자유무역구,개방,상해자유무역구,개방,중국...","시범구,개방,외자지분비율,외자기업,진일보,시범적,중국상해자유무역시범구,격려,공업정보..."


In [17]:
db_save(df, DB_NAME, '키워드')

362 건 저장완료..
